<h2 style="text-align: center;"><strong>Metode Numerik</strong><br /><strong><span style="color: blue;">Metode Numerik di Statistik Sederhana</span></strong><br /><img style="height: 200px;" src="images/MetNum.png" alt="" /></h2>
<h2 id="(C)-Taufik-Sutanto---2019" style="text-align: center;">(C) Taufik Sutanto - 2020</h2>
<h2 id="tau-data-Indonesia-~-https://tau-data.id/MFDS-NM-01/" style="text-align: center;">tau-data Indonesia ~ <a href="https://tau-data.id/MFDS-NM-02/" target="_blank"><span style="color: #0009ff;">https://tau-data.id/MFDS-NM-02/</span></a></h2>

# Quick Review

Sebelumnya kita telah membahas:
* Konversi basis bilangan dan error yang muncul karenanya di sistem floating point.
* Error mutlak & absolut
* Significant digit, truncation, rounding, dan cancellation error.

## Pada kesempatan kali ini kita akan melihat lebih jauh pengaruh error-error tersebut ketika rumus statistika sederhana seperti Rata-Rata dan Variansi dihitung dengan cara biasa dan solusi apa yang bisa kita lakukan agar mendapatkan hasil perhitungan yang lebih baik.

In [1]:
# Fungsi menghitung Significant digit
import math

def sigDigit(real, approx):
    rel_error = abs(real - approx)/abs(real)
    return int( -math.log10( rel_error / 0.5 ) )

# Tujuan

Peserta memahami lebih mendalam bagaimana pemrograman Statistika/Data Science (bahkan yang paling sederhana sekalipun) membutuhkan pengetahuan metode/analisa numerik yang baik. Dengan memahami materi di lesson ini peserta diharapkan mulai memahami konsep programming di Big Data dengan konsep dasar yang kuat dan baik.

# Formula awal Rata-Rata dan Variansi Sampel

* Rata-rata sample: $\bar{x}=\frac{1}{N}\sum_{i=1}^{N}{x_i}$
* Variansi sample: $\sigma^2 = \frac{1}{N-1} \sum_{i=1}^{N}{(x_i-\bar{x})^2}$


## Apa makna/filosofi rumus sederhana ini?

# Outline: Studi Empiris (Eksperimen Numerik)

1. Generate contoh data sample (kasus sederhana)
2. Implementasi (well-known) modul (sebagai benchmark sederhana)
3. Implementasi Naif
4. Evaluasi
5. Pendekatan yang lebih baik.

# Generate data sample untuk studi kasus

1. $M = 10^{12}$
2. X ~ U[0,1] dengan panjang $10^6$
3. $X = \{x+M, x \in X\}$

### Dengan cara ini maka seharunya rata-rata akan mendekati $M$ dan variansi seharusnya sangat kecil mendekati 0. 
## Why?

# Elementary Statistics/Statistical Mathematics

<img alt="" src="images/uniform_mean_var.png" />

image source: https://image2.slideserve.com/4702922/uniform-distribution-mean-variance-l.jpg

In [2]:
import numpy as np
np.random.seed(88) # Biasakan menggunakan SEED pada percobaan random/Stokhastik

X = np.random.uniform(0, 1, 10)
Y = X+1
X[0], X, Y

(0.6475510493530234,
 array([0.64755105, 0.50714969, 0.52834138, 0.8962852 , 0.69999119,
        0.7142971 , 0.71733838, 0.22281946, 0.17515452, 0.45684149]),
 array([1.64755105, 1.50714969, 1.52834138, 1.8962852 , 1.69999119,
        1.7142971 , 1.71733838, 1.22281946, 1.17515452, 1.45684149]))

In [3]:
# Plot distribusinya
import matplotlib.pyplot as plt

X = np.random.uniform(0, 1, 3000)
count, bins, ignored = plt.hist(X, 10, density=True)
plt.plot(bins, np.ones_like(bins), linewidth=2, color='r')
plt.show()

<Figure size 640x480 with 1 Axes>

# Our Sample Data

In [4]:
# Hati-hati menggunakan memory yang cukup besar dan komputasi yang cukup besar (lama)!
# Rubah ke nilai yang lebih kecil jika menggunakan laptop/netbook 
# yang Spec-nya Low-Med.
M = 10**12
X = np.random.uniform(0, 1, 10**6) + M
X[0]

1000000000000.2643

# Solusi Benchmark (Numpy)

In [5]:
realMean = np.mean(X)
realVar = np.var(X)
print("sample mean:         ", realMean)
print( "sample variance:     ", realVar)
#print( "Standard Deviation:     ", np.std(X))
# Notice seberapa cepat nilai-nilai ini dihitung? Bandingkan nanti jika kita menggunakan looping.

sample mean:          1000000000000.5
sample variance:      0.08320963338752091


# Nope hasil diatas tidak tepat! Numpy Var by default menghitung Variansi Populasi bukan sample.

* Baca di dokumentasinya disini: https://numpy.org/doc/stable/reference/generated/numpy.var.html
* Dengan mudah di sesuaikan dengan cara berikut:

In [6]:
realMean = np.mean(X)
realVar = np.var(X, ddof=1)
print("sample mean:         ", realMean)
print( "sample variance:     ", realVar)

sample mean:          1000000000000.5
sample variance:      0.08320971659723751


# Solusi Standard (Naive 2-pass)

* Rata-rata sample: $\bar{x}=\frac{1}{N}\sum_{i=1}^{N}{x_i}$
* Variansi sample: $\sigma^2 = \frac{1}{N-1} \sum_{i=1}^{N}{(x_i-\bar{x})^2}$

In [7]:
# Hati-hati komputasi cukup lama!
M = 10**6
sum_ = 0.0 # ingat juggling variable di kuliah pertama
for i in range(M):
    sum_ += X[i]

rerata = sum_/M
print('Significant digit Rata-rata =', sigDigit(realMean, rerata))

var_ = 0.0
for i in range(M):
    var_ += (X[i]-rerata)**2

var_ = var_/(M-1)
print('Significant digit Variansi=', sigDigit(realVar, var_))

Significant digit Rata-rata = 12
Significant digit Variansi= 0


# Mengapa errornya besar sekali?

* Cancellation
* Plus impractical, kenapa?. Algoritma Two Pass tidak cocok untuk data streaming, misal ketika menangani Velocity di Big Data. 

# Hal ini muncul di beberapa kasus nyata, misal di: 

* computing regression coefficients 
* computing Pearson’s correlation coefficient.
* signal processing.

## How to improve?

# Perbaikan #01: Yang biasanya dilakukan Seorang Programmer untuk menangani 2 pass, agar bisa untuk handle streaming analytics (velocity)

<img alt="" src="images/1_pass_var.png" />

## Sehingga perhitungan variansi menjadi:

In [8]:
# Generate ulang X, just to make sure
X = np.random.uniform(0, 1, 10**6) + 10**12

def onePassVar(X):
    sum_, sumsq_ = 0.0, 0.0
    N = len(X)
    for x in X:
        sum_ += x
        sumsq_ += x**2
    rerata = sum_/N
    return (sumsq_ - N*rerata**2)/(N-1)

print('Significant digit Variansi=', sigDigit(realVar, onePassVar(X)))

Significant digit Variansi= -14


# Pelajaran Penting, secara algoritma sepertinya Elegan bisa berakibat buruk secara numerik!

## Algoritma ini hanya cocok untuk data dengan skala dan ukuran kecil.

# Cara paling sederhana menggunakan Sifat Statistika:

## Shifted Variance: Ketika suatu data di translasikan maka variance tidak berubah.

### ingat hikmah kuliah pertama? (terkait normaslisasi dan standarisasi data)

In [9]:
# Hati-hati komputasi cukup lama di komputer dengan spec low-mid
# Generate ulang X, just to make sure
X = np.random.uniform(0, 1, 10**6) + 10**12

X = X-10**12 # Shifting
N = len(X)

sum_ = 0.0 # ingat juggling variable di kuliah pertama
for x in X:
    sum_ += x
rerata = sum_/N

var_ = 0.0
for x in X:
    var_ += (x-rerata)**2
var_ = var_/(N-1)
print('Significant digit Variansi=', sigDigit(realVar, var_))

Significant digit Variansi= 2


# Mengapa hanya dengan "shifting"/menggeser meningkatkan akurasi begitu besar?

# Alternative: Metode Welford

<p><img alt="" src="images/welford.png" /></p>

[*]. Welford, B. P. (1962). Note on a method for calculating corrected sums of squares and products. Technometrics, 4(3), 419-420.
- Penjelasan lebih lanjut: https://math.stackexchange.com/questions/2798082/how-is-welfords-algorithm-derived

In [10]:
# kita harus kembalikan dahulu nilai X
M = 10**12
X = np.random.uniform(0, 1, 10**6) + M

def welford(x_array):
    k = 0
    for x in x_array:
        k += 1
        if k == 1:
            M = x
            S = 0
        else:
            Mnext = M + (x - M) / k
            S = S + (x - M)*(x - Mnext)
            M = Mnext
    return S/(k-1)

print('Significant digit Variansi=', sigDigit(realVar, welford(X)))

Significant digit Variansi= 2


# Mengapa Lebih baik (the logic behind Welford method)

* Single Pass, cocok untuk streaming analytics
* Pembuktian/penjelasan lebih lanjut:
 - https://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/
 - https://alessior.wordpress.com/2017/10/09/onlinerecursive-variance-calculation-welfords-method/

# Jika ingin mendalami lebih lanjut:

Comparison of Several Algorithms for Computing Sample Means and Variances
Robert F. Ling
Journal of the American Statistical Association
Vol. 69, No. 348 (Dec., 1974), pp. 859-866 (8 pages)
Published By: Taylor & Francis, Ltd.
DOI: 10.2307/2286154
https://www.jstor.org/stable/2286154

# End of Modul MFDS-NM-02

<hr />